# Importing libraries

In [7]:
# import entrezpy.esearch.esearcher as esearcher
# import entrezpy.efetch.efetcher as efetcher

In [8]:
# import ensembl_rest
import requests
import cobra

# Loading SBML model

In [9]:
mitocore_mouse = cobra.io.read_sbml_model("Models/Mitocore_mouse.xml")

# Loading list of genes to test

In [10]:
# A file in .txt format with one gene symbol per line
txtfile = "genelist.txt"

In [11]:
# Creating list of genes to look for into the model
with open(txtfile, 'r') as genefile:
    gene_list = genefile.readlines()
gene_list = [gene.split('\n')[0] for gene in gene_list]

# Fetching synonyms

In [12]:
# NCBI API : Not working well
# def fetch_list_of_synonyms(gene_symbol):
#     print(f"Getting synonyms of {gene_symbol}")
#     es = esearcher.Esearcher('esearcher', 'email')
#     a = es.inquire({'db':'Gene', 'term':gene_symbol, 'retmax' : '10', 'rettype' : 'uilist'})
#     syn = [a.get_result().uids]
#     print(syn)
#     print(len(syn))
#     return syn

In [13]:
# Ensembl API : Missing some synonyms and no EC number available
# def get_synonyms(gene_symbol, dict_syn):
#     dict_syn[gene_symbol] = {}

#     request_synonyms = ensembl_rest.xref_name(
#             species='human',
#             name=gene_symbol
#         )
    
#     try:
#         request_id = ensembl_rest.symbol_lookup(
#                 species='human',
#                 symbol=gene_symbol
#             )
        
#         dict_syn[gene_symbol]['ensembl_id'] = request_id['id']

#     except:
#         print("No Ensembl ID found")
#         dict_syn[gene_symbol]['ensembl_id'] = ""
    
#     for dict_db in request_synonyms:
#         if dict_db['dbname'] == "HGNC":
#             dict_syn[gene_symbol]['synonyms'] = dict_db['synonyms']

# request = ensembl_rest.symbol_lookup(
#             species='human',
#             symbol="ACOX2"
#         )


In [14]:
# KEGG API, first step : operation FIND to look for KEGG entry access corresponding to a gene's symbol
# Get the entry for hsa organism (homo sapiens)
def get_entry_access(gene_symbol):
    url = "http://rest.kegg.jp/find/genes/"
    r = requests.get(url=url+gene_symbol)

    if r.status_code==200:
        info = str(r.content).split('\'')[1].split('\\n')[0].split('\\t')

        if info[0].split(':')[0] == 'hsa':
            entry_access = info[0]
            return entry_access
        
        print(f"Warning : no hsa entry access found for gene {gene_symbol}")
        return 0
    
    else: 
        print(f"Warning : gene {gene_symbol} is not found in kegg")

    return 0

In [15]:
# KEGG API, second step : operation GET to look for all info from a KEGG entry access corresponding to a gene's symbol
def get_all_info(entry_access, gene_symbol):
    if entry_access == 0:
        return 0
    
    url = "http://rest.kegg.jp/get/"
    r = requests.get(url=url+entry_access)

    if r.status_code==200:
        lines = str(r.content).split('\'')[1].split('\\n')
        dblink = False
        symbols = []
        ECNumber = 0
        ensembl_id = 0

        for line in lines:
            keyword = line.split(' ')[0]

            if keyword == "SYMBOL":
                symbols_line = line.split(' ')

                for s in symbols_line:
                    if s != 'SYMBOL' and s != '':
                        symbols.append(s)

            elif keyword == "ORTHOLOGY":
                if 'EC:' in line:
                    print(line)
                    ECNumber = line.split(':')[1].split(']')[0]
                else:
                    ECNumber = 0

            elif keyword == "DBLINKS":
                dblink = True

            elif dblink:
                if line.split(' ')[-2] == 'Ensembl:':
                    ensembl_id = line.split(' ')[-1]

                    return symbols, ECNumber, ensembl_id
    
    else:
        print(f"Warning : gene {gene_symbol} is not found in kegg")
    
    return 0


In [16]:
# Example for an access entry corresponding to ACOX2
get_all_info('hsa:8309', 'ACOX2')

ORTHOLOGY   K10214  3alpha,7alpha,12alpha-trihydroxy-5beta-cholestanoyl-CoA 24-hydroxylase [EC:1.17.99.3]


(['ACOX2,', 'BCOX,', 'BRCACOX,', 'BRCOX,', 'CBAS6,', 'THCCox'],
 '1.17.99.3',
 'ENSG00000168306')

In [17]:
# dict_synonyms = {}

# for gene in gene_list:
#     print(f"Getting synonyms and Ensembl id for {gene}")
#     get_synonyms(gene, dict_synonyms)

In [18]:
# dict_synonyms = {}
# for gene in gene_list:
#         # dict_synonyms[gene] = fetch_list_of_synonyms(gene)
#         fetch_list_of_synonyms(gene)


In [19]:
def create_dict_synonyms(genes_list, dict_synonyms = {}):
    
    for gene in genes_list:
        # print(gene)
        entry = get_entry_access(gene)
        # print(entry)
        info = get_all_info(entry, gene)

        if info != 0:
            dict_synonyms[gene] = {'Synonyms': [syn.upper() for syn in info[0]], 'ECN': info[1], 'ensembl_id': info[2]}
        else:
            dict_synonyms[gene] = {'Synonyms': [gene.upper()], 'ECN': 0, 'ensembl_id': 0}
    
    return dict_synonyms

In [20]:
# Takes ~7min for 146 genes
dict_synonyms = create_dict_synonyms(gene_list)

ORTHOLOGY   K10214  3alpha,7alpha,12alpha-trihydroxy-5beta-cholestanoyl-CoA 24-hydroxylase [EC:1.17.99.3]
ORTHOLOGY   K01895  acetyl-CoA synthetase [EC:6.2.1.1]
ORTHOLOGY   K08618  a disintegrin and metalloproteinase with thrombospondin motifs 2 [EC:3.4.24.14]
ORTHOLOGY   K00011  aldehyde reductase [EC:1.1.1.21]
ORTHOLOGY   K00461  arachidonate 5-lipoxygenase [EC:1.13.11.34]
ORTHOLOGY   K03766  beta-1,3-N-acetylglucosaminyltransferase 5 [EC:2.4.1.206]
ORTHOLOGY   K15717  prostamide/prostaglandin F2alpha synthase [EC:1.11.1.20]
ORTHOLOGY   K07232  glutathione-specific gamma-glutamylcyclotransferase [EC:4.3.2.7]
ORTHOLOGY   K08638  carboxypeptidase X1 [EC:3.4.17.-]
ORTHOLOGY   K08784  HtrA serine peptidase 1 [EC:3.4.21.-]
ORTHOLOGY   K16816  patatin-like phospholipase domain-containing protein 2 [EC:3.1.1.3]
ORTHOLOGY   K08650  kallikrein 8 [EC:3.4.21.118]
ORTHOLOGY   K14738  metalloreductase STEAP2 [EC:1.16.1.-]
ORTHOLOGY   K10699  ubiquitin-activating enzyme E1-like protein 2 [EC:6.2.1

# Model's Reactions

In [21]:
reactions_list = [reaction.id for reaction in mitocore_mouse.reactions]

In [22]:
def create_dict_model(model, reactions, final_dict = {}):
    reactions_wo_info = []
    ECn = []
    ids = []
    genes = []

    for reaction in reactions:
        notes = model.reactions.get_by_id(reaction).notes

        if ('EC Number' in notes) and (notes['EC Number'] != 'Unknown'):
            ECn = notes['EC Number'].split()
            ECn = [number for number in ECn if number not in ('or', 'and')]

        if ('GENE_ASSOCIATION' in notes) and (notes['GENE_ASSOCIATION'] != 'Unknown'):
            ids = notes['GENE_ASSOCIATION'].replace('(', '')
            ids = ids.replace(')', '')
            ids = ids.split()
            ids = [id for id in ids if id not in ('or', 'and')]

        if ('GENE_LIST' in notes) and (notes['GENE_LIST'] != 'Unknown'):
            genes = notes['GENE_LIST'].replace('(','')
            genes = genes.replace(')', '')
            genes = genes.split()
            genes = [gene.upper() for gene in genes if gene not in ('or', 'and')]

        if ECn == [] and id == [] and genes == []:
            reactions_wo_info.append(reaction)
            print(f"No information found, {reaction} is not added to dictionnary")
        else:
            final_dict[reaction] = {'ECN': ECn, 'ensembl_id': ids, 'genes': genes}

    return final_dict, reactions_wo_info


In [23]:
output = create_dict_model(mitocore_mouse, reactions_list)
names_to_check = output[1]
dict_model = output[0]

In [24]:
dict_model

{'EX_2hb_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_ac_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_acac_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_akg_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_ala_B_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_ala_L_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_arg_L_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_argsuc_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_asn_L_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_asp_L_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_bhb_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_bilirub_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_biomass_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_but_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_chol_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_cit_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_citr_L_e': {'ECN': [], 'ensembl_id': [], 'genes': []},
 'EX_c

In [25]:
dict_synonyms

{'ACOX2': {'Synonyms': ['ACOX2,',
   'BCOX,',
   'BRCACOX,',
   'BRCOX,',
   'CBAS6,',
   'THCCOX'],
  'ECN': '1.17.99.3',
  'ensembl_id': 'ENSG00000168306'},
 'ACSS2': {'Synonyms': ['ACSS2,',
   'ACAS2,',
   'ACECS,',
   'ACS,',
   'ACSA,',
   'ACECS1,',
   'DJ1161H23.1'],
  'ECN': '6.2.1.1',
  'ensembl_id': 'ENSG00000131069'},
 'ADAMTS2': {'Synonyms': ['ADAMTS2,',
   'ADAM-TS2,',
   'ADAMTS-2,',
   'ADAMTS-3,',
   'EDSDERMS,',
   'NPI,',
   'PC_I-NP,',
   'PCI-NP,',
   'PCINP,',
   'PCPNI,',
   'PNPI'],
  'ECN': '3.4.24.14',
  'ensembl_id': 'ENSG00000087116'},
 'AEBP1': {'Synonyms': ['AEBP1,', 'ACLP'],
  'ECN': 0,
  'ensembl_id': 'ENSG00000106624'},
 'AKR1B10': {'Synonyms': ['AKR1B10,',
   'AKR1B11,',
   'AKR1B12,',
   'ALDRLN,',
   'ARL-1,',
   'ARL1,',
   'HIS,',
   'HSI'],
  'ECN': '1.1.1.21',
  'ensembl_id': 'ENSG00000198074'},
 'ALOX5': {'Synonyms': ['ALOX5,', '5-LO,', '5-LOX,', '5LPG,', 'LOG5'],
  'ECN': '1.13.11.34',
  'ensembl_id': 'ENSG00000012779'},
 'ANKRD1': {'Synonyms': 

In [26]:
# Checks model's reactions with no information on Genes, EC number and Ensembl ID

for gene in dict_synonyms:
    for synonym in dict_synonyms[gene]['Synonyms']:
        if synonym in names_to_check:
            print(f"{gene} found in model as {synonym}")

In [27]:
final_list = {}

for reaction in dict_model:
    # checks synonyms
    for gene_model in dict_model[reaction]['genes']:
        for gene_check in dict_synonyms:
            if gene_model in dict_synonyms[gene_check]['Synonyms']:
                if gene_check in final_list:
                    if reaction not in final_list[gene_check]:
                        final_list[gene_check].append(reaction)
                else:
                    final_list[gene_check] = [reaction]
                print(f"{gene_check} found in model as {reaction}({gene_model})")
    
    # checks EC number
    for gene_check in dict_synonyms:
        if dict_synonyms[gene_check]['ECN'] in dict_model[reaction]['ECN']:
            if gene_check in final_list:
                if reaction not in final_list[gene_check]:
                    final_list[gene_check].append(reaction)
            else:
                final_list[gene_check] = [reaction]
            print(f"{gene_check} found in model as {reaction}({dict_synonyms[gene_check]['ECN']})")

    # checks Ensembl ID
    for gene_check in dict_synonyms:
        if dict_synonyms[gene_check]['ensembl_id'] in dict_model[reaction]['ensembl_id']:
            if gene_check in final_list:
                if reaction not in final_list[gene_check]:
                    final_list[gene_check].append(reaction)
            else:
                final_list[gene_check] = [reaction]
            print(f"{gene_check} found in model as {reaction}({dict_synonyms[gene_check]['ensembl_id']})")    


G6PC found in model as G6PPer(3.1.3.9)
G6PC found in model as G6PPer(ENSG00000131482)
ALDOC found in model as FBA(4.1.2.13)
ALDOC found in model as FBA(ENSG00000109107)
PCK2 found in model as PEPCK(4.1.1.32)
DLST found in model as AKGDm(2.3.1.61)
DLST found in model as AKGDm(ENSG00000119689)
NDUFS1 found in model as CI_MitoCore(ENSG00000023228)
COX6A2 found in model as CIV_MitoCore(ENSG00000156885)
COX7A2L found in model as CIV_MitoCore(ENSG00000115944)
ATP5C1 found in model as CV_MitoCore(ENSG00000165629)
PCK2 found in model as PEPCKm(4.1.1.32)
PCK2 found in model as PEPCKm(ENSG00000100889)
ACAA1 found in model as MTPC16_MitoCore(2.3.1.16)
ACAA1 found in model as MTPC14_MitoCore(2.3.1.16)
ACAA1 found in model as r0724(2.3.1.16)
ACAA1 found in model as r0634(2.3.1.16)
ACAA1 found in model as r0732(2.3.1.16)
ACAA1 found in model as r0287(2.3.1.16)
ACAT2 found in model as ACACT1rm(2.3.1.9)
ACAA1 found in model as ACACT10m(2.3.1.16)
ACSS2 found in model as ACCOALm(6.2.1.1)
SLC25A21 found 

In [28]:
final_list

{'G6PC': ['G6PPer'],
 'ALDOC': ['FBA'],
 'PCK2': ['PEPCK', 'PEPCKm'],
 'DLST': ['AKGDm', '2OXOADOXm'],
 'NDUFS1': ['CI_MitoCore'],
 'COX6A2': ['CIV_MitoCore'],
 'COX7A2L': ['CIV_MitoCore'],
 'ATP5C1': ['CV_MitoCore'],
 'ACAA1': ['MTPC16_MitoCore',
  'MTPC14_MitoCore',
  'r0724',
  'r0634',
  'r0732',
  'r0287',
  'ACACT10m'],
 'ACAT2': ['ACACT1rm'],
 'ACSS2': ['ACCOALm', 'ACSm', 'ACS'],
 'SLC25A21': ['2OXOADPTmB_MitoCore',
  '2OXOADPTmC_MitoCore',
  '2AMADPTmB_MitoCore',
  '2AMADPTmC_MitoCore',
  'ORNDC'],
 'CKB': ['CKc'],
 'UCP3': ['HtmB_MitoCore']}

dict_synonym = {symbol : {synonym: [], ECN: "", ID: ""}}

dict_model = {reaction : {genes: [], ECN: [], ID:[]}}